In [12]:
import pandas as pd
data=pd.read_csv('train.txt',delimiter=';', header=None) #delimiter or sep
data.columns = ["Text","Emotions"]
data.head()

,Text,Emotions
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Text      16000 non-null  object
 1   Emotions  16000 non-null  object
dtypes: object(2)
memory usage: 250.1+ KB


In [14]:
data.describe()

,Text,Emotions
count,16000,16000
unique,15969,6
top,i feel on the verge of tears from weariness i ...,joy
freq,2,5362


In [15]:
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.utils import to_categorical


In [16]:
Texts= data['Text']
lables=data['Emotions']

In [17]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(Texts)
sequence=tokenizer.texts_to_sequences(Texts)
print(sequence)

[[1, 138, 2, 678], [1, 39, 100, 59, 7, 14, 493, 4, 14, 3495, 552, 31, 59, 60, 127, 147, 75, 1479, 3, 21, 1254], [16, 3059, 6, 1148, 4, 285, 1, 2, 494, 437], [1, 23, 164, 7, 664, 26, 5, 4157, 1, 58, 46, 8, 12, 21, 71, 29, 5, 3496], [1, 23, 7, 1064], [72, 47, 7, 6, 55, 520, 318, 327, 157, 160, 8, 19], [72, 47, 328, 34, 7400, 34, 195, 7401, 887, 3, 72, 2474, 1383, 6, 158, 1884, 18, 1, 116, 2, 13, 14, 454], [1, 2, 28, 438, 26, 77, 28, 6, 1685, 34, 28, 759, 28, 6, 192, 266, 373], [1, 20, 47, 24, 5259, 15, 214, 1, 2, 8, 5259, 98, 5260, 134, 3, 131, 6, 1037, 4158], [1, 2, 665, 93], [1, 2, 13, 1, 20, 4, 79, 5, 732, 1, 92, 543, 303, 83], [1, 38, 2, 8, 482, 21, 6, 529, 439, 3, 8, 1, 39, 919, 4, 20, 68, 839, 9, 1885, 3060], [1, 69, 12, 89, 5, 5261, 51, 9, 192, 4, 2, 609], [1, 2, 338, 455, 1, 92, 31, 4159], [1, 20, 4160, 1686, 24, 5, 987, 258, 18, 28, 6, 888, 7402, 1612, 190, 4, 163, 51, 4, 224, 11, 5, 3061, 9, 77, 3, 24, 95, 1770, 9, 76, 3497, 206, 200, 6, 2475, 3498, 22, 122, 6, 55, 503], [1, 38

In [18]:
sequences = tokenizer.texts_to_sequences(Texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences,maxlen = max_length)

In [19]:
padded_sequences

array([[   0,    0,    0, ...,  138,    2,  678],
       [   0,    0,    0, ...,    3,   21, 1254],
       [   0,    0,    0, ...,    2,  494,  437],
       ...,
       [   0,    0,    0, ...,    3,  101, 1331],
       [   0,    0,    0, ...,  339,    8,   42],
       [   0,    0,    0, ...,   25, 3585,   12]], dtype=int32)

In [20]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
lables=le.fit_transform(lables)
lables

array([4, 4, 0, ..., 2, 0, 4])

In [21]:
from tensorflow.keras.utils import to_categorical
lables=to_categorical(lables)
lables

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.]])

In [22]:
one_hot_labels = keras.utils . to_categorical(lables)

In [24]:
X_train,X_test,y_train,y_test=train_test_split(padded_sequences,lables,test_size=0.2)


In [25]:


model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 21s 46ms/step - accuracy: 0.3826 - loss: 1.5273 - val_accuracy: 0.7409 - val_loss: 0.8031
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 19s 44ms/step - accuracy: 0.8788 - loss: 0.4128 - val_accuracy: 0.8256 - val_loss: 0.5427
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 15s 36ms/step - accuracy: 0.9842 - loss: 0.0638 - val_accuracy: 0.8256 - val_loss: 0.6165
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 15s 38ms/step - accuracy: 0.9961 - loss: 0.0232 - val_accuracy: 0.8216 - val_loss: 0.6089
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 15s 39ms/step - accuracy: 0.9977 - loss: 0.0131 - val_accuracy: 0.8288 - val_loss: 0.6283
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 15s 37ms/step - accuracy: 0.9965 - loss: 0.0150 - val_accuracy: 0.8266 - val_loss: 0.6791
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 15s 37ms/step - accuracy: 0.9979 - loss: 0.0101 - val_accuracy: 0.8216 - val_loss: 0.7001
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 14s 36ms/step - accuracy: 0.9977 - loss: 0.0097 - 

In [34]:
import numpy as np
input_text = "i find myself in the odd position of feeling supportive of"

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = le.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
['love']
